# Volcado Tabular de Vencimientos Explained con Otros Campos

In [70]:
from datetime import date
import pandas as pd
import requests

## Rutas

In [71]:
rutas = {
    "explained": "https://ms-tesoreria-analytics-staging.interconecta2.cl/v1/settlements/swaps/explained",
    "headers": "https://ms-tesoreria-analytics-staging.interconecta2.cl/operations/all",
}

## SDK

Estas son funciones que utilizan los endpoints adecuados para obtener la data necesaria para el volcado.

In [72]:
def get_explained(fecha: date, ruta: str):
    result = requests.get(f"{ruta}/{fecha}")
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

In [73]:
def get_headers(ruta: str):
    result = requests.get(ruta)
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

## Procedimiento

Se define la fecha de proceso.

In [74]:
fecha_proceso = date(2025, 7, 29)
f_proc = fecha_proceso.isoformat().replace("-", "")

Se obtienen los explained a fecha de proceso.

In [75]:
explained = get_explained(fecha_proceso, rutas["explained"])

Se obtienen los encabezados de la cartera vigente.

In [76]:
headers = get_headers(rutas["headers"])

Vemos un registro de los explained.

In [77]:
explained[0]

{'dealNumber': '2305',
 'legNumber': 1,
 'tipoPata': 'FIXED_RATE',
 'recPay': 'A',
 'fechaInicial': '2025-01-29',
 'fechaFinal': '2025-07-29',
 'fechaPago': '2025-07-29',
 'nocional': 5500000000.0,
 'amortizacion': 0.0,
 'interes': 107016249.99999948,
 'amortEsFlujo': True,
 'flujo': 107016249.99999948,
 'monedaNocional': 'CLP',
 'valorTasa': 0.0387,
 'tipoTasa': 'LinAct360',
 'fraccionAgno': '(fechaFinal - fechaInicial) / 360',
 'formulaInteres': 'nocional * valorTasa * fraccionAgno'}

Se observa que no tiene los campos de portfolio (gestión o contable) ni la estrategia contable. Se los agregamos con la data de los headers.

In [78]:
df_explained = pd.DataFrame.from_dict(explained)

Vemos la estructura de un header.

In [79]:
headers["5000"]

{'trade_date': {'fecha': '2022-11-29'},
 'deal_number': '5000',
 'counterparty': {'name': 'BANCO SANTANDER-SANTIAGO',
  'rut': {'rut': 97036000, 'dv': 'K'},
  'other': None},
 'portfolio': 'BANCA',
 'hedge_accounting': 'COB_TASA_INT_ACTIVOS',
 'product': 'SWAP_MONE',
 'currency_pair': 'CLFCLP',
 'price': None,
 'settlement_mechanism': 'C',
 'other': {'regulatory_portfolio': 'COBERTURA_VALOR_RAZONABLE'}}

In [80]:
def generate_new_columns(row):
    # Notar que headers se toma por closure
    operacion = headers.get(row.dealNumber)
    if operacion is None:
        operacion = {"dummy": 1}
    if operacion.get("other") is None:
        regulatory_portfolio = None
    else:
        regulatory_portfolio = operacion.get("other").get("regulatory_portfolio")
    return pd.Series({
        'portfolio': operacion.get("portfolio"),
        'hedgeAccounting': operacion.get("hedge_accounting"),
        'product': operacion.get("product"),
        'settlementMechanism': operacion.get("settlement_mechanism"),
        'regulatoryPortfolio': regulatory_portfolio,
    })

In [81]:
df_explained_ok = df_explained.join(df_explained.apply(
    generate_new_columns,
    axis=1,
))
del df_explained_ok["fraccionAgno"]
del df_explained_ok["formulaValorTasaEquivalente"]
del df_explained_ok["formulaInteres"]
# TRY - EXCEPT
del df_explained_ok["formulaInteresMonedaPago"]
del df_explained_ok["formulaAmortizacionMonedaPago"]

In [82]:
df_explained_ok

,dealNumber,legNumber,tipoPata,recPay,fechaInicial,fechaFinal,fechaPago,nocional,amortizacion,interes,...,valorIndiceFx,amortizacionMonedaPago,interesMonedaPago,flujoMonedaPago,codigoIndiceTasa,portfolio,hedgeAccounting,product,settlementMechanism,regulatoryPortfolio
0,2305,1,FIXED_RATE,A,2025-01-29,2025-07-29,2025-07-29,5.500000e+09,0.0,1.070162e+08,...,NaN,NaN,NaN,NaN,NaN,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
1,2305,2,OVERNIGHT_INDEX,P,2025-01-29,2025-07-29,2025-07-29,-5.500000e+09,0.0,-1.399231e+08,...,NaN,NaN,NaN,NaN,NaN,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
2,2711,1,FIXED_RATE,A,2025-01-29,2025-07-29,2025-07-29,3.000000e+09,0.0,4.389250e+07,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
3,2711,2,OVERNIGHT_INDEX,P,2025-01-29,2025-07-29,2025-07-29,-3.000000e+09,0.0,-7.632167e+07,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
4,3517,1,FIXED_RATE_MCCY,A,2025-01-29,2025-07-29,2025-07-29,2.000000e+09,0.0,1.910556e+07,...,957.82,0.000000e+00,1.994692e+04,1.994692e+04,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
5,3517,2,OVERNIGHT_INDEX_MCCY,P,2025-01-29,2025-07-29,2025-07-29,-2.000000e+09,0.0,-5.315013e+04,...,957.82,0.000000e+00,-5.315013e+04,-5.315013e+04,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
6,3991,1,OVERNIGHT_INDEX_MCCY,A,2025-01-29,2025-07-29,2025-07-29,2.800000e+09,0.0,7.441019e+04,...,957.82,0.000000e+00,7.441019e+04,7.441019e+04,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
7,3991,2,FIXED_RATE_MCCY,P,2025-01-29,2025-07-29,2025-07-29,-2.800000e+09,0.0,-5.800044e+07,...,957.82,0.000000e+00,-6.055464e+04,-6.055464e+04,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
8,3992,1,FIXED_RATE_MCCY,A,2025-01-29,2025-07-29,2025-07-29,2.800000e+09,0.0,5.814122e+07,...,957.82,0.000000e+00,6.070162e+04,6.070162e+04,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
9,3992,2,OVERNIGHT_INDEX_MCCY,P,2025-01-29,2025-07-29,2025-07-29,-2.800000e+09,0.0,-7.441019e+04,...,957.82,0.000000e+00,-7.441019e+04,-7.441019e+04,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION


In [83]:
#df_explained_ok.to_csv(f"./{f_proc}_explained_plus_plus.csv")

Ahora vamos a convertir la dataframe en una vista por operación.

In [84]:
import numpy as np

Crear una función helper para agregar los datos correctamente

In [85]:
def aggregate_deal_data(group):
    """
    Aggregate deal data from leg-level to deal-level
    """
    # Get deal-level attributes (should be the same for all legs)
    deal_attrs = {
        'DealNumber': group['dealNumber'].iloc[0],
        'product': group['product'].iloc[0],
        'fechaFinal': group['fechaFinal'].iloc[0],
        'settlementMechanism': group['settlementMechanism'].iloc[0],
        'hedgeAccounting': group['hedgeAccounting'].iloc[0],
        'regulatoryPortfolio': group['regulatoryPortfolio'].iloc[0]
    }
    
    # Separate active (A) and passive (P) legs
    active_legs = group[group['recPay'] == 'A']
    passive_legs = group[group['recPay'] == 'P']
    
    # Process active legs
    if len(active_legs) > 0:
        deal_attrs['pataActivaAmortizacion'] = active_legs['amortizacion'].sum()
        deal_attrs['pataActivaMonedaNocional'] = active_legs['monedaNocional'].iloc[0]  # Assuming same currency for all active legs
        # Get fechaPago from lowest legNumber for active legs
        deal_attrs['pataActivaFechaPagoAmortizacion'] = active_legs.loc[active_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        active_flujo_values = active_legs['flujoMonedaPago'].fillna(active_legs['flujo'])
        deal_attrs['pataActivaFlujo'] = active_flujo_values.sum() if not active_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataActivaAmortizacion'] = np.nan
        deal_attrs['pataActivaMonedaNocional'] = np.nan
        deal_attrs['pataActivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataActivaFlujo'] = np.nan
    
    # Process passive legs
    if len(passive_legs) > 0:
        deal_attrs['pataPasivaAmortizacion'] = passive_legs['amortizacion'].sum()
        deal_attrs['pataPasivaMonedaNocional'] = passive_legs['monedaNocional'].iloc[0]  # Assuming same currency for all passive legs
        # Get fechaPago from lowest legNumber for passive legs
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = passive_legs.loc[passive_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        passive_flujo_values = passive_legs['flujoMonedaPago'].fillna(passive_legs['flujo'])
        deal_attrs['pataPasivaFlujo'] = passive_flujo_values.sum() if not passive_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataPasivaAmortizacion'] = np.nan
        deal_attrs['pataPasivaMonedaNocional'] = np.nan
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataPasivaFlujo'] = np.nan
    
    return pd.Series(deal_attrs)

Agrupamos por deal number

In [86]:
df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)

C:\Users\bencl\AppData\Local\Temp\ipykernel_40880\4184180563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)


Aplicar conversiones de nombres de producto

In [87]:
# Apply product name conversions
product_mapping = {
    'SWAP_ICP': 'Swap Promedio Cámara'.encode('utf-8').decode('utf-8'),
    'SWAP_MONE': 'Swap Moneda',
    'SWAP_TASA': 'Swap Tasas'
}
df_deal_level['product'] = df_deal_level['product'].map(product_mapping).fillna(df_deal_level['product'])

Y calcular monto neto a compensar en caso de modalidad compensación

In [88]:
df_deal_level['netoACompensar'] = np.nan
mask_compensation = df_deal_level['settlementMechanism'] == 'C'

df_deal_level.loc[mask_compensation, 'netoACompensar'] = (
    df_deal_level.loc[mask_compensation, 'pataActivaFlujo'].fillna(0) + 
    df_deal_level.loc[mask_compensation, 'pataPasivaFlujo'].fillna(0)
)

Y reordenamos las columnas

In [89]:
# Reorder columns to include the new netoACompensar column
column_order = [
    'DealNumber', 'product', 'fechaFinal', 'settlementMechanism', 'hedgeAccounting', 
    'regulatoryPortfolio', 'pataActivaAmortizacion', 'pataActivaMonedaNocional', 
    'pataActivaFechaPagoAmortizacion', 'pataActivaFlujo', 'pataPasivaAmortizacion',
    'pataPasivaMonedaNocional', 'pataPasivaFechaPagoAmortizacion', 'pataPasivaFlujo',
    'netoACompensar'
]

df_deal_level = df_deal_level[column_order]

Y vemos los resultados:

In [90]:
print("Deal-level dataframe shape:", df_deal_level.shape)
print("\nFirst few rows:")
print(df_deal_level.head())

print("\nDataframe info:")
print(df_deal_level.info())

print("\nSample of deal-level data:")
print(df_deal_level.head(10))

Deal-level dataframe shape: (10, 15)

First few rows:
  DealNumber               product  fechaFinal settlementMechanism  \
0       2305  Swap Promedio Cámara  2025-07-29                   C   
1       2711  Swap Promedio Cámara  2025-07-29                   C   
2       3517  Swap Promedio Cámara  2025-07-29                   C   
3       3991  Swap Promedio Cámara  2025-07-29                   C   
4       3992  Swap Promedio Cámara  2025-07-29                   C   

  hedgeAccounting regulatoryPortfolio  pataActivaAmortizacion  \
0              NO         NEGOCIACION                     0.0   
1              NO         NEGOCIACION                     0.0   
2              NO         NEGOCIACION                     0.0   
3              NO         NEGOCIACION                     0.0   
4              NO         NEGOCIACION                     0.0   

  pataActivaMonedaNocional pataActivaFechaPagoAmortizacion  pataActivaFlujo  \
0                      CLP                      2025-07

In [91]:
df_deal_level

,DealNumber,product,fechaFinal,settlementMechanism,hedgeAccounting,regulatoryPortfolio,pataActivaAmortizacion,pataActivaMonedaNocional,pataActivaFechaPagoAmortizacion,pataActivaFlujo,pataPasivaAmortizacion,pataPasivaMonedaNocional,pataPasivaFechaPagoAmortizacion,pataPasivaFlujo,netoACompensar
0,2305,Swap Promedio Cámara,2025-07-29,C,NO,NEGOCIACION,0.0,CLP,2025-07-29,1.070162e+08,0.0,CLP,2025-07-29,-1.399231e+08,-3.290681e+07
1,2711,Swap Promedio Cámara,2025-07-29,C,NO,NEGOCIACION,0.0,CLP,2025-07-29,4.389250e+07,0.0,CLP,2025-07-29,-7.632167e+07,-3.242917e+07
2,3517,Swap Promedio Cámara,2025-07-29,C,NO,NEGOCIACION,0.0,CLP,2025-07-29,1.994692e+04,0.0,CLP,2025-07-29,-5.315013e+04,-3.320321e+04
3,3991,Swap Promedio Cámara,2025-07-29,C,NO,NEGOCIACION,0.0,CLP,2025-07-29,7.441019e+04,0.0,CLP,2025-07-29,-6.055464e+04,1.385555e+04
4,3992,Swap Promedio Cámara,2025-07-29,C,NO,NEGOCIACION,0.0,CLP,2025-07-29,6.070162e+04,0.0,CLP,2025-07-29,-7.441019e+04,-1.370857e+04
5,5161,Swap Moneda,2025-07-29,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,254339465.6,CLP,2025-07-29,3.091927e+08,-7216.0,CLF,2025-07-29,-3.092214e+08,-2.871270e+04
6,5771,Swap Tasas,2025-07-29,C,NO,NEGOCIACION,0.0,USD,2025-07-29,1.571487e+05,0.0,USD,2025-07-29,-1.858950e+05,-2.874632e+04
7,6299,Swap Moneda,2025-07-29,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.0,CLF,2025-07-29,1.707105e+08,0.0,CLP,2025-07-29,-3.552282e+08,-1.845177e+08
8,6905,Swap Moneda,2025-07-29,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.0,CLF,2025-07-29,3.369286e+07,0.0,CLP,2025-07-29,-8.604818e+07,-5.235532e+07
9,7153,Swap Promedio Cámara,2025-07-29,C,NO,NEGOCIACION,0.0,CLP,2025-07-29,1.926896e+08,0.0,CLP,2025-07-29,-1.857161e+08,6.973527e+06


In [ ]:
#df_deal_level.to_csv(f"./{f_proc}_deal_level.csv", index=False, encoding='utf-8')
#df_deal_level.to_excel(f"./{f_proc}_deal_level.xlsx", index=False)
df_deal_level.to_excel(rf"C:\Users\bencl\OneDrive - palace.cl\Documents\Palace\Banco Internacional\Treasury PM\06 Pruebas\expiry_complement_{f_proc}.xlsx", index=False)
